In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from copy import deepcopy

# convert series to supervised learning
def series_to_supervised(data, n_in = 1, n_out = 1, dropnan = True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()

	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]

	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]

	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names

	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

# getting data
features = ['SWTP Total Influent Flow', 'SWTP Plant 2 Influent Flow', 'Wilsons Gauge Height (ft)', 'James Gauge Height (ft)', 
            'SWTP Plant 1 Influent Flow', 'Fire 168 Hour Rainfall Aggregate', 'AT&T 168 Hour Rainfall Aggregate', 
            'Field 168 Hour Rainfall Aggregate', 'Springfield Plateau Aquifer Depth to Water Level (ft)', 
            'Ozark Aquifer Depth to Water Level (ft)', 'Month', 'Hour', 'Week', 'Year']

dataset = pd.read_csv("Imputed Data.csv", usecols=features)
values = dataset.values

# linear transformation of each feature from [min, max] to [0, 1]
scaler = MinMaxScaler()
scaled = scaler.fit_transform(values)

# reframing data into a forecasting problem
numBackward = 25
numForward = 73
reframed = series_to_supervised(scaled, numBackward, numForward)            #(x, 73) for 72 values in the future, x is a hyperparameter

# dropping columns we do not need
numCols = len(dataset.columns)
undesiredCols = [x for x in range(numCols * numBackward + 1, numCols * (numBackward + numForward))]
reframed.drop(reframed.columns[undesiredCols], axis = 1, inplace =True)     #everthing after var1(t) must be dropped

# splitting into training and testing sets
values = reframed.values
n_train_hours = int(44260 * 0.8)                            # taking 80% of the data to train on
train = deepcopy(values[:n_train_hours:, :])
test = deepcopy(values[n_train_hours:, :])

# separating into X and Y datasets
train_X, train_y = train[:, :-1], train[:, -1]              # [all rows, up to last col, i.e. var1(t)] and [all rows, just last col, i.e. var1(t)]
test_X, test_y = test[:, :-1], test[:, -1]

# changing input to be 3D for the model
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

#### Main Hyperparameters to tune: numBackward, n_epochs, LSTM units, Dense units

In [2]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM
from kerastuner.tuners import BayesianOptimization
import os

def build_model(hp):
    model = Sequential()
    model.add(LSTM(units=hp.Int('units',min_value=1, max_value=60, step=2), 
               activation='tanh', input_shape=(train_X.shape[1], train_X.shape[2])))
    model.add(Dense(units=hp.Int('units',min_value=1, max_value=60, step=2)))
    model.add(Dense(1))
    model.compile(loss='mse', metrics=['mse'], optimizer=keras.optimizers.Adam(
        hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4, 1e-5])))
    return model

n_epochs = 20
bayesian_opt_tuner = BayesianOptimization(
    build_model,
    objective='mse',
    max_trials=5,
    executions_per_trial=1,
    directory=os.path.normpath('C:/Users/natha/Desktop/Undergrad/Spring2022/MTH 596 PIC Math/Project - Group 2/Project/Forecasting'),
    project_name='keras_tuner_attempt1',
    overwrite=True)

bayesian_opt_tuner.search(train_X, train_y, epochs=n_epochs,
     #validation_data=(X_test, y_test)
     validation_split=0.2,verbose=1)


bayes_opt_model_best_model = bayesian_opt_tuner.get_best_models(num_models=1)
model = bayes_opt_model_best_model[0]

path = "C:\\Users\\natha\\Desktop\\Undergrad\\Spring2022\\MTH 596 PIC Math\\Project - Group 2\\Project\\Forecasting\\keras_tuner_attempt1\\model"
model.save(path)

Trial 5 Complete [00h 01m 01s]
mse: 0.0003998268221039325

Best mse So Far: 0.0003998268221039325
Total elapsed time: 00h 04m 57s
INFO:tensorflow:Oracle triggered exit


In [10]:
modelAttempt = keras.models.load_model(path)